# Extract and summarize information with LLMs 

## Purpose of the notebook

In this notebook you'll find how to run LLM model to extract useful data from human and public contribution.

## Requirements

Be sure LangChain and transformers from huggingface are installed. It is recommanded to install huggingface transformers package from source repo.  

`!pip install langchain`  
`!pip install git+https://github.com/huggingface/transformers`

## Load the LLM model

In [1]:
import langchain
langchain.__version__

'0.1.4'

In [2]:
from langchain_community.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, pipeline
from langchain_community.llms import VLLM, VLLMOpenAI
from langchain.chains import LLMChain
from pprint import pprint
import json

import gc
import torch

gc.collect()
torch.cuda.empty_cache()

/home/machine_learning/.anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define the prompt

In [10]:
system_message = """###
Context: Le champs d'application est l'analyse de sondage et de consultation publique.
Ton: Formel.
Audience: Chercheurs ou analystes de sondages.
###
Objectif:
1. Réalise le résumé de <<< TEXT >>> sous forme de synthèse des faits et propositions principaux.
2. Liste étape par étape les items principaux. Un item est un fait ou une proposition. Un item n'est pas un exemple.
3. Ecrit les items obligatoirement soit sous forme d'une phrase complète avec un sujet, un verbe et un complément, soit commence par un verbe à l'infinitif d'ordre.
    - Ne fait pas de sous item.
    - Fusionne en un seul item les items très proches.
    - Précise si un item est une proposition ("label":"proposition") ou un fait ("label":"fact").
    - Précise si l'item est syntaxiquement négatif ou positif. Un item négatif contient une négation, par exemple: ["ne", "n'", "ne pas", "ne plus de", "non"].

###
Réponse: Strict Format JSON. Exemple: ```
{"summary": "Les chats sont très agiles, ils retombent toujours sur leurs pattes. Cependant, ils n'ont pas neuve vies. L'on devrait mieux prendre soin d'eux.",
"items":[
        {"name": "Les chats retombent sur leurs pattes", "negative":"false", "label":"fact"},
        {"name": "Les chats n'ont pas neuve vies", "negative":"true", "label":"fact"},
        {"name": "Prendre mieux soin des animaux", "negative":"false", "label":"proposition"}
]}
```
"""

question = """
Résume et extrait les items principaux de <<< TEXT >>>.
Précise si l'item est syntaxiquement négatif ou non.
Répond avec au format JSON.
<<< {input} >>>
JSON:"""

In [11]:
template = """<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{question}<|im_end|>
<|im_start|>assistant
"""

## Run locally

In [6]:
model_name = "TheBloke/NeuralBeagle14-7B-AWQ"

tokenizer = AutoTokenizer.from_pretrained(model_name)
vllm_kwargs = {
    "quantization": "awq",
    "max_model_len": 3000,
    "max_context_len_to_capture":2000,
    "gpu_memory_utilization":0.9,
    # "enforce_eager":True,
}
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
llm = VLLM(
    model=model_name,
    trust_remote_code=True,  # mandatory for hf models
    top_k=10,
    top_p=0.95,
    temperature=0.2,
    dtype="auto",
    repetition_penalty=1.3,
    streamer=streamer,
    vllm_kwargs=vllm_kwargs,
)

2024-04-03 16:19:04,548	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


WARNING 04-03 16:19:04 config.py:177] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 04-03 16:19:04 llm_engine.py:72] Initializing an LLM engine with config: model='TheBloke/NeuralBeagle14-7B-AWQ', tokenizer='TheBloke/NeuralBeagle14-7B-AWQ', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=3000, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, seed=0)
INFO 04-03 16:19:08 weight_utils.py:164] Using model weights format ['*.safetensors']
INFO 04-03 16:20:08 llm_engine.py:322] # GPU blocks: 704, # CPU blocks: 2048
WARNING 04-03 16:20:08 cache_engine.py:100] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.
INFO 04-03 16:20:08 model_runner.py:632] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the mode

In [27]:
input = """
réduire drastiquement la fraude fiscale. Imposer les grands groupes (GAFA) qui ne le sont pas suffisamment Renforcer la taxe sur les transactions  financières.
"""

question_ = question.format(input=input)
template_ = template.format(question=question_, system_message=system_message)

length_prompt = len(template_)
print(f'{length_prompt=}')

length_prompt=1782


In [28]:
prompt = PromptTemplate(
    template=template, input_variables=["system_message", "question"]
)
llm_chain = LLMChain(prompt=prompt, llm=llm)

result = llm_chain.invoke(
    {
        "system_message": system_message,
        "question": question.format(input=input),
        "stop": ["<|im_end|>"],
    }
)

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.84s/it]


In [29]:
print(result["text"])

{"summary": "Propositions pour réduire la fraude fiscale et imposer les grands groupes (GAFA): 1. Réduire drastiquement la fraude fiscale, 2. Imposer les grands groupes non payants, 3. Renforcer la taxe sur les transactions financières.",
"items": [
    {
        "name": "Réduire drastiquement la fraude fiscale",
        "negative": "false",
        "label": "fact"
    },
    {
        "name": "Imposer les grands groupes non payants",
        "negative": "false",
        "label": "fact"
    },
    {
        "name": "Renforcer la taxe sur les transactions financières",
        "negative": "false",
        "label": "proposition"
    ]
]}

